In [1]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-6c700f89-63cb-3e48-347e-67b2584aa709)


In [1]:
#Importing the libraries
import numpy as np
from tqdm import tqdm
import os
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, CuDNNLSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
os.listdir()

['Text Generation.ipynb',
 'LICENSE',
 'internet_archive_scifi_v3.txt',
 '.ipynb_checkpoints',
 'README.md',
 'scifi-stories-text-corpus.zip',
 '.git',
 'weights-improvement-01-2.6655.hdf5',
 'weights-improvement-02-2.4036.hdf5']

In [5]:
!unzip scifi-stories-text-corpus.zip

Archive:  scifi-stories-text-corpus.zip
  inflating: internet_archive_scifi_v3.txt  


In [3]:
!sudo chmod 777 internet_archive_scifi_v3.txt
!stat -c "%a %n" internet_archive_scifi_v3.txt

777 internet_archive_scifi_v3.txt


In [4]:
#Load the raw text and convert them to lowercase
filename = "internet_archive_scifi_v3.txt"
text_corpus = open(filename).read()
len_text = len(text_corpus)
text_corpus = text_corpus[0:len_text//100]
text_corpus = text_corpus.lower()

In [5]:
len(text_corpus)

1493263

In [6]:
#Create mapping from characters to integers
chars = sorted(list(set(text_corpus)))
chars_to_int = dict((char, integer) for integer, char in enumerate(chars))

In [7]:
#Print the unique characters present in the corpus
print(chars_to_int.keys())

dict_keys(['i', 'n', '!', '-', 'k', 'p', 'w', 'f', 's', '#', 'a', 'y', 'u', 'v', ':', ',', 't', 'z', 'g', 'q', ';', "'", 'd', '.', '"', '(', 'b', 'c', 'h', 'x', ')', 'j', 'r', '?', 'e', 'o', 'l', 'm', ' '])


In [8]:
#Basic information about the dataset
n_chars = len(text_corpus)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  1493263
Total Vocab:  39


In [9]:
#Prepare the dataset of input to output pairs encoded as integers
seq_length = 100
X_data = []
y_data = []
for i in tqdm(range(0, n_chars - seq_length, 1)):
    input_sequence = text_corpus[i:i + seq_length]
    output_sequence = text_corpus[i + seq_length]
    X_data.append([chars_to_int[char] for char in input_sequence])
    y_data.append(chars_to_int[output_sequence])
n_patterns = len(X_data)
print ("Total Patterns: ", n_patterns)

100%|██████████| 1493163/1493163 [00:17<00:00, 87341.19it/s] 

Total Patterns:  1493163


In [10]:
#Reshape X to be [samples, time steps, features]
X = np.reshape(X_data, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(y_data)

In [11]:
# define the LSTM model
model = Sequential()
model.add(CuDNNLSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(CuDNNLSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

W0813 08:02:10.340634 140034485409536 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0813 08:02:10.359927 140034485409536 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0813 08:02:10.939487 140034485409536 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0813 08:02:11.158742 140034485409536 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0813 08:02:11.168537 

In [12]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [13]:
model.fit(X, y, epochs=5, batch_size=128, callbacks=callbacks_list)

W0813 08:02:17.750810 140034485409536 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
1493163/1493163 [==============================] - 289s 194us/step - loss: 2.3316

Epoch 00001: loss improved from inf to 2.33164, saving model to weights-improvement-01-2.3316.hdf5
Epoch 2/5
1493163/1493163 [==============================] - 286s 191us/step - loss: 1.9742

Epoch 00002: loss improved from 2.33164 to 1.97416, saving model to weights-improvement-02-1.9742.hdf5
Epoch 3/5
1493163/1493163 [==============================] - 286s 192us/step - loss: 1.8572

Epoch 00003: loss improved from 1.97416 to 1.85723, saving model to weights-improvement-03-1.8572.hdf5
Epoch 4/5
1493163/1493163 [==============================] - 287s 192us/step - loss: 1.7903

Epoch 00004: loss improved from 1.85723 to 1.79033, saving model to weights-improvement-04-1.7903.hdf5
Epoch 5/5
1493163/1493163 [==============================] - 286s 192us/step - loss: 1.7437

Epoch 00005: loss improved from 1.79033 to 1.74371, saving model to weights-improvement-05-1.7437.hdf5


In [14]:
filename = "weights-improvement-05-1.7437.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [15]:
ints_to_chars = dict((integer, char) for integer, char in enumerate(chars))

In [28]:
start = np.random.randint(0, len(X_data)-1)
pattern = X_data[start]
print ("Seed:")
print ("\"", ''.join([ints_to_chars[value] for value in pattern]), "\"")
# generate characters
for i in range(500):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = ints_to_chars[index]
    seq_in = [ints_to_chars[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
" ," she muttered. "that must be just the aim of our nose. i didn't feel a course-change." she tugged  "
and stroec and stroected the street of the said and stroected to the same of the street of the same and strorised the street of the street of the street of the street was a sharp of the street of the street of the street was a sharp of the street of the street of the street was a sharp of the street of the street of the street was a sharp of the street of the street of the street was a sharp of the street of the street of the street was a sharp of the street of the street of the street was a sha
Done.


In [23]:
import sys

In [29]:
pattern = X_data[start]

In [30]:
pattern

[7,
 2,
 0,
 31,
 20,
 17,
 0,
 25,
 33,
 32,
 32,
 17,
 30,
 17,
 16,
 9,
 0,
 2,
 32,
 20,
 13,
 32,
 0,
 25,
 33,
 31,
 32,
 0,
 14,
 17,
 0,
 22,
 33,
 31,
 32,
 0,
 32,
 20,
 17,
 0,
 13,
 21,
 25,
 0,
 27,
 18,
 0,
 27,
 33,
 30,
 0,
 26,
 27,
 31,
 17,
 9,
 0,
 21,
 0,
 16,
 21,
 16,
 26,
 4,
 32,
 0,
 18,
 17,
 17,
 24,
 0,
 13,
 0,
 15,
 27,
 33,
 30,
 31,
 17,
 8,
 15,
 20,
 13,
 26,
 19,
 17,
 9,
 2,
 0,
 31,
 20,
 17,
 0,
 32,
 33,
 19,
 19,
 17,
 16,
 0,
 13]